In [1]:
!pip install  optuna xgboost imbalanced-learn

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 605.4 kB/s eta 0:01:59
   ---------------------------------------- 0.5/72.0 MB 605.4 kB/s eta 0:01:59
   ---------------------------------------- 0.8/72.0 MB 570.2 kB/s eta 0:02:05
    --------------------------------------- 1.0/72.0 MB 642.9 kB/s eta 0:01:51
    --------------------------------------- 1.0/72.0 MB 642.9 kB/s eta 0:01:51
    --------------------------------------- 1.0/72.0 MB 642.9 kB/s eta 0:01:51
    --------------------------------------- 1.3/72.0 MB 631.1 kB/s eta 0:01:52
    --------------------------------------- 1.6/72.0 MB 697.6 kB/s eta 0:01:41
   - -------------

In [3]:
!pip install dagshub

  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 3.4 MB/s eta 0:00:01
   ------------------------------ --------- 1.6/2.0 MB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.2 MB/s  0:00:00
   ---------------------------------------- 0.0/14.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.1 MB 4.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/14.1 MB 4.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.1 MB 4.2 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.1 MB 3.9 MB/s eta 0:00:03
   ----------- ---------------------------- 3.9/14.1 MB 4.1 MB/s eta 0:00:03
   -------------- ------------------------- 5.0/14.1 MB 4.1 MB/s eta 0:00:03
   ---------------- ----------------------- 5.

In [5]:
!pip install mlflow


In [1]:

import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

Accessing as Vaibha3246

Initialized MLflow to track repo "Vaibha3246/influence_mirror"

Repository Vaibha3246/influence_mirror initialized!

In [2]:
# set or create an experiment
mlflow.set_experiment("exp_5 ml_algo_with_hp_tunning")


<Experiment: artifact_location='mlflow-artifacts:/adb82bab710d416190b0fea77cabca06', creation_time=1760159677050, experiment_id='3', last_update_time=1760159677050, lifecycle_stage='active', name='exp_5 ml_algo_with_hp_tunning', tags={}>

In [12]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import optuna
import mlflow
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier


In [12]:
!pip install seaborn

In [4]:
import pandas as pd
df=pd.read_csv('preprocessing.csv').dropna(subset=['text_clean'])

In [ ]:
# -------------------------
# Step 1: Clean target
# -------------------------
df['sentiment_numeric'] = df['sentiment_numeric'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['sentiment_numeric'])

# -------------------------
# Step 2: Select features
# -------------------------
numeric_cols = [col for col in df.columns if col not in [
    'video_id', 'category', 'text', 'text_clean', 'sentiment',
    'dominant_emotion', 'published_at', 'sentiment_numeric'
]]

X_numeric = df[numeric_cols]
y = df['sentiment_numeric']

# Scale numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Train/test split
X_train_num, X_test_num, y_train, y_test, train_idx, test_idx = train_test_split(
    X_numeric_scaled, y, df.index, test_size=0.2, random_state=42, stratify=y
)

# -------------------------
# Step 3: TF-IDF for text
# -------------------------
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_text = tfidf.fit_transform(df.loc[train_idx, 'text_clean'])
X_test_text = tfidf.transform(df.loc[test_idx, 'text_clean'])

# Combine numeric + text
X_train = sp.hstack([X_train_text, sp.csr_matrix(X_train_num)])
X_test = sp.hstack([X_test_text, sp.csr_matrix(X_test_num)])

# -------------------------
# Step 4: Apply SMOTE
# -------------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# -------------------------
# Step 5: Define Optuna objective
# -------------------------
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2.0),
        "objective": "multi:softmax",       # <-- returns class labels directly
        "num_class": len(np.unique(y)),
        "random_state": 42,
        "n_jobs": -1
    }

    model = XGBClassifier(**params)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc

# -------------------------
# Step 6: Run Optuna tuning
# -------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

# -------------------------
# Step 7: Train best XGBoost model
# -------------------------
best_params = study.best_params
best_model = XGBClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)
y_pred = best_model.predict(X_test)

# -------------------------
# Step 8: Log metrics + model in MLflow
# -------------------------
with mlflow.start_run(run_name="XGBoost_TFIDF_SMOTE_Optuna_Best"):
    # Log algorithm name
    mlflow.log_param("algorithm", "XGBoost")

    # Log hyperparameters
    mlflow.log_params(best_params)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", (1, 3))
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("imbalance_method", "SMOTE")

    # Log accuracy
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))

    # Log full classification report metrics
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix: XGBoost Best Model")
    plt.savefig("confusion_matrix_xgboost_best.png")
    mlflow.log_artifact("confusion_matrix_xgboost_best.png")
    plt.close()

    # Save the model locally
    model_path = "xgboost_best_model.pkl"
    joblib.dump(best_model, model_path)

    # Log the model file as an artifact 
    mlflow.log_artifact(model_path)

print("MLflow run logged successfully for XGBoost ")

[I 2025-10-25 14:36:49,867] A new study created in memory with name: no-name-f78ec68f-ecc8-46de-a924-e9c4ff0dc8f3
[I 2025-10-25 14:37:21,123] Trial 0 finished with value: 0.7709727800089246 and parameters: {'n_estimators': 278, 'max_depth': 14, 'learning_rate': 0.1669680477571992, 'subsample': 0.6342909054687238, 'colsample_bytree': 0.7518088865596301, 'gamma': 4.66996267152681, 'min_child_weight': 8, 'reg_alpha': 0.5795900930377961, 'reg_lambda': 1.5402383787202092}. Best is trial 0 with value: 0.7709727800089246.
[I 2025-10-25 14:38:07,513] Trial 1 finished with value: 0.7735385988398037 and parameters: {'n_estimators': 361, 'max_depth': 4, 'learning_rate': 0.1040101830539823, 'subsample': 0.6692722993399883, 'colsample_bytree': 0.9856367649453861, 'gamma': 0.057876241636348746, 'min_child_weight': 7, 'reg_alpha': 0.40953036863099024, 'reg_lambda': 0.6386518932019354}. Best is trial 1 with value: 0.7735385988398037.
[I 2025-10-25 14:38:41,804] Trial 2 finished with value: 0.724899598

🏃 View run XGBoost_TFIDF_SMOTE_Optuna_Best at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3/runs/f2cd05872e404228ba045d98dcc29f8c
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3
MLflow run logged successfully for XGBoost 


In [7]:
import xgboost